In [10]:
# Import required libraries
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
import numpy as np
from os import urandom
from IPython.display import display
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os

In [11]:
# AES key
key = os.urandom(16)  # 128-bit key

In [12]:
# AES block size
block_size_bytes = 16

In [13]:
# Plaintext (arbitrary length)
plaintext = b'A un amigo perdido. Ya no me debes recordar en el pais extranjero. Tambien a mi se me escapa tu imagen lejana.'  

In [14]:
print("Plaintext :", plaintext)

Plaintext : b'A un amigo perdido. Ya no me debes recordar en el pais extranjero. Tambien a mi se me escapa tu imagen lejana.'


In [15]:
print("Plaintext (hex):", plaintext.hex())

Plaintext (hex): 4120756e20616d69676f207065726469646f2e205961206e6f206d65206465626573207265636f7264617220656e20656c20706169732065787472616e6a65726f2e2054616d6269656e2061206d69207365206d652065736361706120747520696d6167656e206c656a616e612e


In [16]:
print(' '.join(format(b, '08b') for b in plaintext))

01000001 00100000 01110101 01101110 00100000 01100001 01101101 01101001 01100111 01101111 00100000 01110000 01100101 01110010 01100100 01101001 01100100 01101111 00101110 00100000 01011001 01100001 00100000 01101110 01101111 00100000 01101101 01100101 00100000 01100100 01100101 01100010 01100101 01110011 00100000 01110010 01100101 01100011 01101111 01110010 01100100 01100001 01110010 00100000 01100101 01101110 00100000 01100101 01101100 00100000 01110000 01100001 01101001 01110011 00100000 01100101 01111000 01110100 01110010 01100001 01101110 01101010 01100101 01110010 01101111 00101110 00100000 01010100 01100001 01101101 01100010 01101001 01100101 01101110 00100000 01100001 00100000 01101101 01101001 00100000 01110011 01100101 00100000 01101101 01100101 00100000 01100101 01110011 01100011 01100001 01110000 01100001 00100000 01110100 01110101 00100000 01101001 01101101 01100001 01100111 01100101 01101110 00100000 01101100 01100101 01101010 01100001 01101110 01100001 00101110


In [17]:
# 1. Apply PKCS7 padding
padder = padding.PKCS7(block_size_bytes * 8).padder()
padded_plaintext = padder.update(plaintext) + padder.finalize()

In [18]:
print(' '.join(format(b, '08b') for b in padded_plaintext))

01000001 00100000 01110101 01101110 00100000 01100001 01101101 01101001 01100111 01101111 00100000 01110000 01100101 01110010 01100100 01101001 01100100 01101111 00101110 00100000 01011001 01100001 00100000 01101110 01101111 00100000 01101101 01100101 00100000 01100100 01100101 01100010 01100101 01110011 00100000 01110010 01100101 01100011 01101111 01110010 01100100 01100001 01110010 00100000 01100101 01101110 00100000 01100101 01101100 00100000 01110000 01100001 01101001 01110011 00100000 01100101 01111000 01110100 01110010 01100001 01101110 01101010 01100101 01110010 01101111 00101110 00100000 01010100 01100001 01101101 01100010 01101001 01100101 01101110 00100000 01100001 00100000 01101101 01101001 00100000 01110011 01100101 00100000 01101101 01100101 00100000 01100101 01110011 01100011 01100001 01110000 01100001 00100000 01110100 01110101 00100000 01101001 01101101 01100001 01100111 01100101 01101110 00100000 01101100 01100101 01101010 01100001 01101110 01100001 00101110 00000010 0

### 2. Create a new AES-ECB encryptor and encrypt the plaintext

In this step, we take the **padded plaintext** and encrypt it using **AES in ECB mode**.

```python
cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
encryptor = cipher.encryptor()
ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()
print("Ciphertext (hex):", ciphertext.hex())


In [19]:
# 2. Create a new AES-ECB encryptor
cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
encryptor = cipher.encryptor()
ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()

In [20]:
print("Ciphertext (hex):", ciphertext.hex())

Ciphertext (hex): 40ff827d4359c829682a91f2ee31d286ee591e3bf2a6422b03312e205676bb8d573087caede42c532f48caa8eb40800fca4eb32f2c6cf299f998ec80b4f6a7a948312513745c99464f46c96fca275d14a358d3e0eb1423e44380316ba9153f66e85e270d5f9c9ca0a05a796aee1fc47f


In [21]:
print(' '.join(format(b, '08b') for b in ciphertext))

01000000 11111111 10000010 01111101 01000011 01011001 11001000 00101001 01101000 00101010 10010001 11110010 11101110 00110001 11010010 10000110 11101110 01011001 00011110 00111011 11110010 10100110 01000010 00101011 00000011 00110001 00101110 00100000 01010110 01110110 10111011 10001101 01010111 00110000 10000111 11001010 11101101 11100100 00101100 01010011 00101111 01001000 11001010 10101000 11101011 01000000 10000000 00001111 11001010 01001110 10110011 00101111 00101100 01101100 11110010 10011001 11111001 10011000 11101100 10000000 10110100 11110110 10100111 10101001 01001000 00110001 00100101 00010011 01110100 01011100 10011001 01000110 01001111 01000110 11001001 01101111 11001010 00100111 01011101 00010100 10100011 01011000 11010011 11100000 11101011 00010100 00100011 11100100 01000011 10000000 00110001 01101011 10101001 00010101 00111111 01100110 11101000 01011110 00100111 00001101 01011111 10011100 10011100 10100000 10100000 01011010 01111001 01101010 11101110 00011111 11000100 0

### Step-by-step explanation

**`Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())`**  
- Creates a cipher object using AES with the given key.  
- `modes.ECB()` specifies the Electronic Codebook (ECB) mode, the simplest AES mode.  
- `backend=default_backend()` tells the library to use the system’s default cryptographic backend (usually OpenSSL).

**`encryptor = cipher.encryptor()`**  
- Creates a one-time encryption context.  
- Each encryptor can only be used once for a single encryption operation.

**`encryptor.update(padded_plaintext) + encryptor.finalize()`**  
- `update(padded_plaintext)` processes the plaintext in 16-byte blocks.  
- `finalize()` completes the encryption and returns any remaining bytes.  
- The result is the ciphertext, which is exactly the same length as the padded plaintext.

**`print("Ciphertext (hex):", ciphertext.hex())`**  
- Prints the ciphertext as a hexadecimal string so it is human-readable.

---

### Notes for students

- ECB mode encrypts each 16-byte block independently, so identical blocks in the plaintext will produce identical ciphertext blocks.  
- ECB is not recommended for real applications, but it is excellent for teaching the basic encryption process.  
- Padding ensures the plaintext length is a multiple of the AES block size (16 bytes), which is required by ECB mode.


In [22]:
# 3. Decrypt
decryptor = cipher.decryptor()
decrypted_padded = decryptor.update(ciphertext) + decryptor.finalize()

### Step 3: Decrypt the ciphertext

**`decryptor = cipher.decryptor()`**  
- Creates a new decryption context from the same cipher object.  
- Each decryptor can only be used once for a single decryption operation.  

**`decrypted_padded = decryptor.update(ciphertext) + decryptor.finalize()`**  
- `update(ciphertext)` processes the ciphertext in 16-byte blocks and decrypts it.  
- `finalize()` completes the decryption and returns any remaining bytes.  
- The result is the **padded plaintext**, which still includes the PKCS#7 padding added during encryption.

---

**Notes for students:**

- The decrypted output is not yet the original plaintext; it still contains padding.  
- AES-ECB decrypts each block independently, matching the way encryption was done.  
- A new decryptor must be created each time you perform decryption; you cannot reuse the same object for multiple operations.


In [23]:
# 4. Remove padding
unpadder = padding.PKCS7(block_size_bytes * 8).unpadder()
recovered = unpadder.update(decrypted_padded) + unpadder.finalize()
print("Recovered plaintext:", recovered)

Recovered plaintext: b'A un amigo perdido. Ya no me debes recordar en el pais extranjero. Tambien a mi se me escapa tu imagen lejana.'


### Step 4: Remove padding

**`unpadder = padding.PKCS7(block_size_bytes * 8).unpadder()`**  
- Creates an **unpadding context** for AES.  
- AES block size = 128 bits (16 bytes), so we multiply by 8 to get bits for PKCS#7.

**`recovered = unpadder.update(decrypted_padded) + unpadder.finalize()`**  
- `update(decrypted_padded)` removes the padding bytes from the decrypted data.  
- `finalize()` completes the unpadding process.  
- The result is the **original plaintext**, exactly as it was before encryption.

**`print("Recovered plaintext:", recovered)`**  
- Displays the recovered plaintext to verify that encryption and decryption worked correctly.

---

**Notes for students:**

- Padding is necessary because AES operates on fixed 16-byte blocks.  
- PKCS#7 padding adds extra bytes to make the plaintext a multiple of the block size.  
- Removing the padding restores the original message.  
- This step is essential when encrypting plaintexts of arbitrary length.
